# Reto 3: El problema final

Este reto consiste en predecir si una imagen contiene o no signos de retinopatía diabética (DR).
Para ello se proporciona un conjunto de datos que contiene características extraídas de imágenes.
Las características extraídas son:
+ [0] Evaluación de la calidad de la imagen , donde 0 = mala calidad 1 = calidad suficiente.
+ [1] El resultado binario de la evaluación previa, donde 1 indica una anormalidad retiniana severa y 0 su falta.
+ [2-7] Los resultados de la detección de microaneurismas (MA). Cada valor de característica representa el número de MAs encontrados en los niveles de confianza alfa = 0.5, ... , 1, respectivamente.
+ [8-15] contienen la misma información que [2-7] para los exudados. <br>
Sin embargo, ya que los exudados están representados por un conjunto de puntos en lugar del número de
píxeles de las lesiones, estas características se normalizan dividiendo los número de lesiones por  el diámetro de la ROI para compensar las diferentes tamaños de las imágenes.
+ [16] La distancia euclidiana del centro dela mácula y el centro del disco óptico para proporcionar información importante con respecto a la condición del paciente. Esta característica también se normaliza con el diámetro de la ROI
+ [17] El diámetro del disco óptico.
+ [18] El resultado binario de la clasificación basada en AM/FM.
+ [19] Etiqueta de clase. 1 = contiene signos de DR

El conjunto de datos, *retinopatia_reto3.csv*, consiste en 592 muestras de imágenes; cada uno de ellos representado por 19 características.

Para evaluar las propuestas se utilizará un conjunto de datos que se mantendrá oculto hasta después de la entrega

### Requisitos
+ **Se debe utilizar algún tipo de reducción de la dimensión o aprendizaje de variedades**
+ A continuación se debe entrenar un clasificador o combinación de clasificadores con las características transformadas según el requisito anterior.
+ Se debe entregar un cuaderno Jupyter con el nombre de los participantes.<br>
  *Por ejemplo*:   **Cuesta_Hinton.ipynb**
+ El cuaderno entregado debe seguir la estructura y reglas de este cuaderno

### Competición
+ Todos los cuadernos entregados se subirán al repo de GitHub y se ejecutarán en Binder, donde ya estará en conjunto de test que estaba oculto.
+ El resultado que se obtenga será la puntuación del reto.
+ **Importante** Es muy fácil asegurarte de que tu código funciona:
    1. Agrupa todo tu código en una única celda
    2. Copialo en el cuaderno del reto que hay en Binder
    3. Ejecuta el cuaderno 
    
### Plazo: lunes 16 de nov. a las 6 am.
Es decir, incluye toda la noche del domingo 15 de nov.
 
> "The final problem", A. C. Doyle (Strand Magazine, diciembre 1983), 
es el relato corto en el que Sherlock Holmes se enfrenta al Prof. Moriarty en las cataratas Reichenbach.

---
    [ES] Código de Alfredo Cuesta Infante para 'Reconocimiento de Patrones'
       @ Master Universitario en Visión Artificial, 2020, URJC (España)
    [EN] Code by Alfredo Cuesta-Infante for 'Pattern Recognition'
       @ Master of Computer Vision, 2020, URJC (Spain)

    alfredo.cuesta@urjc.es

In [1]:
# NO TOCAR ESTA CELDA
# Conjunto distribuido para el reto

Challange_filename = '../../Datasets/retinopatia_reto3.csv'


In [2]:
# NO TOCAR ESTA CELDA
# El conjunto de test cambiará una vez se cierre la entrega
# Ahora mismo es el mismo conjunto que el de entrenamiento

Test_filename = '../../Datasets/retinopatia_test.csv' #<-- este nombre cambiará después del plazo de entrega

In [3]:
# NO TOCAR ESTA CELDA

#-[1]. Load data from CSV and put all in a single dataframe 'FullSet'

import numpy  as np
import pandas as pd
from matplotlib import pyplot as plt
import sys
sys.path.append('../../MyUtils/')
import MyUtils as my
seed = 1234 #<- random generator seed (comment to get randomness)

#-[2]. Load data from CSV and put all in a single dataframe 'FullSet'

FullSet = pd.read_csv(Challange_filename, header=0)
FullX = FullSet.iloc[:,:-1]
FullY = FullSet.iloc[:,-1]

<table style="width:100%;"> 
 <tr style='background:lime'>
  <td style="text-align:left">
      <h2>Tu código debe empezar a partir de aquí y puede tener tantas celdas como quieras</h2>
      <p> Si quieres, puedes borrar (o convertir en RawNBConvert) las celdas de ejemplo
      <h3>Importante:</h3>
      <p>Tu código debe producir las siguientes variables: </p>
      <p> $\quad \bullet$ <b>clf:</b> el clasificador final con el que se realizará el test<br>
       $\quad \bullet$ <b>X_test:</b> el conjunto de test listo para ser usado por el método <b>predict</b><br>
       $\quad \bullet$ <b>Y_test:</b> es el vector de etiquetas del conjunto de X_test listo para ser usado por el método <b>confusion_matrix</b>
      </p>
  </td>
 </tr>
</table>

In [4]:
nombres = ["Ian de la Oliva"]

Trabajo realizado:
+ Prueba de modelos previa--> Solo voy a trabajar con __Gradient Boosting__.
+ Pasos en cada modelo.
    - Train/test al 0.3 (para test).
    - Escalamos el train. Escalamos test con los parámetros de train.
    - Realizamos __PCA__ con 0.7 de información a retener.
    - Realizamos un grid search sobre los Gradient Boosting con los parámetros de la celda de abajo y 3-cross validation.
+ He realizado 20 veces el paso anterior (pocos minutos) para cada separación. Selecciono el ganador y me quedo con ese estimador. 
+ Después de ejecutar este proceso 3 veces tengo 3 clasificadores:
    - GradientBoostingClassifier(criterion='mae', learning_rate=0.05, max_depth=5,
                             max_features='log2', min_samples_leaf=0.1,
                             min_samples_split=0.31818181818181823,
                             n_estimators=200, subsample=0.95)
    - GradientBoostingClassifier(criterion='mae', learning_rate=0.01, max_depth=8,
                             max_features='sqrt', min_samples_leaf=0.1,
                             min_samples_split=0.390909090909091,
                             n_estimators=400, subsample=0.9)
    - GradientBoostingClassifier(learning_rate=0.075, max_depth=5,
                             max_features='sqrt', min_samples_leaf=0.1,
                             min_samples_split=0.46363636363636374,
                             n_estimators=50, subsample=0.85)
+ Compruebo cada uno de estos estimadores realizando una media de los resultados en test de dividir (y entrenar) el conjunto inicial y realizar el PCA correspondiente. Esto lo hago para tener una __consistencia__ ya que los resultados varian mucho en test segun la division de train test. Seleccionamos el ganador.
+ Por último entreno el ganador sobre todo el conjunto (por ser muy pequeño) para que la explotación sea la mejor.

 # Imports 

In [8]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import PCA

# Separación escalado

# PCA 

# Grid search

# Estudio consistencia top 3 modelos salidos de los distintos Grid Search

# Ganador

In [9]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(FullX)
Y_train = FullY.values.ravel() 

n_components = .7 
pca = PCA(n_components = n_components)
pca.fit(X_train)
X_train = pca.transform(X_train)

clf =  GradientBoostingClassifier(learning_rate=0.075, max_depth=5,
                             max_features='sqrt', min_samples_leaf=0.1,
                             min_samples_split=0.46363636363636374,
                             n_estimators=50, subsample=0.85)

clf.fit(X_train, Y_train)
Y_hat = clf.predict(X_train)
conf_mat = confusion_matrix(Y_train , Y_hat)
N_success  = np.trace(conf_mat)
N_fails = Y_train.shape[0]-N_success
print (nombres,"\n")
print("Confusion matrix:\n")
print(conf_mat,"\n")
print("Outcome:\n")
strlog = "  :) HIT  = %d, (%0.2f%%)"%(N_success, 100*N_success/(N_success+N_fails))
print(strlog)
strlog = "  :( FAIL = %d, (%0.2f%%)"%(N_fails, 100*N_fails/(N_success+N_fails))
print(strlog)

['Ian de la Oliva'] 

Confusion matrix:

[[339 107]
 [223 282]] 

Outcome:

  :) HIT  = 621, (65.30%)
  :( FAIL = 330, (34.70%)


In [10]:
#-- ejemplo de test --

FullSet = pd.read_csv(Test_filename, header=0)
TestX_ = FullSet.iloc[:,:-1]
TestY_ = FullSet.iloc[:,-1]
X_test = scaler.transform(TestX_)
X_test = pca.transform(X_test)
Y_test = TestY_.values.ravel() 
#-la evaluación se realiza en las celdas de abajo

'''RESULTADO: X_test es el dataframe para utilizar en >>> Y_pred = clf.predict() 
   RESULTADO: Y_test es el array con las etiquetas para utilizar en >>> confusion_matrix(Y_test,Y_pred)
'''

'RESULTADO: X_test es el dataframe para utilizar en >>> Y_pred = clf.predict() \n   RESULTADO: Y_test es el array con las etiquetas para utilizar en >>> confusion_matrix(Y_test,Y_pred)\n'

<table style="width:100%;"> 
 <tr style='background:pink'>
  <td style="text-align:left">
      <h2>A partir de aquí ya no se pueden modificar las celdas</h2>
          <h3>Comprueba que:</h3>
          <p> $\quad \bullet$ tu clasificador está almacenado en la variable <b>clf</b><br>
              $\quad \bullet$ tienes el conjunto de test correctamente almacenado en la variable <b>X_test</b><br>
              $\quad \bullet$ tienes las etiquetas del conjunto de test correctamente almacenadas en la variable <b>Y_test</b><br>
          </p>
      
  </td>
 </tr>
</table>

## Test

In [11]:
# NO TOCAR ESTA CELDA

from sklearn.metrics import confusion_matrix

Y_hat = clf.predict(X_test)
conf_mat = confusion_matrix(Y_test , Y_hat)
N_success  = np.trace(conf_mat)
N_fails = Y_test.shape[0]-N_success
#-------------------------------
print (nombres,"\n")
print("Confusion matrix:\n")
print(conf_mat,"\n")
print("Outcome:\n")
strlog = "  :) HIT  = %d, (%0.2f%%)"%(N_success, 100*N_success/(N_success+N_fails))
print(strlog)
strlog = "  :( FAIL = %d, (%0.2f%%)"%(N_fails, 100*N_fails/(N_success+N_fails))
print(strlog)

['Ian de la Oliva'] 

Confusion matrix:

[[339 107]
 [223 282]] 

Outcome:

  :) HIT  = 621, (65.30%)
  :( FAIL = 330, (34.70%)
